In [1]:
from langchain.schema.document import Document
from langchain_chroma import Chroma
from embedding_function import get_embedding_function
from langchain.prompts import ChatPromptTemplate
import argparse
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import huggingface_hub
from langchain_community.embeddings import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


In [2]:
CHROMA_PATH = "chroma"

In [3]:
PROMPT_TEMPLATE = """
Du er en finansiell rådgiver. brug de givne oplysninger til at besvare spørgsmålet så præcist som muligt:
kontekst:
{context}

---
Spørgsmål:
{question}

Svar på dansk, og hvis svaret ikke kan findes i konteksten, sig det tydeligt:
"""



In [4]:

# Get query from user input in Jupyter Notebook
query_text = input("hvad vil du: ")

# Ensure embedding_function uses OllamaEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding_function = embeddings


def query_rag(query_text: str):    # Load the vector database
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
    results = db.similarity_search_with_score(query_text, k=5)
    if not results:
        print("Ingen relevante dokumenter fundet i Chroma.")
        return None

    print("\n📄 --- Chroma Retrieval Results ---")
    for i, (doc, score) in enumerate(results):
        print(f"\nResultat {i+1} (score: {score:.4f}):")
        print(f"Source: {doc.metadata.get('source', 'ukendt')}")
        print(doc.page_content[:500], "..." if len(doc.page_content) > 500 else "")


    # Prepare context for prompt
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    llm = Ollama(model="mistral")
    response_text = llm.invoke(prompt)

    sources = [doc.metadata.get("source", "ukendt") for doc, _ in results]
    formatted_response = f"Svar: {response_text}\n\nKilder:\n\n{sources}"
    print(formatted_response)
    #return response_text



# Call the function to query RAG with the user's input
query_rag(query_text)



📄 --- Chroma Retrieval Results ---

Resultat 1 (score: 0.9259):
Source: Data/Maersk Annual Report 2024.pdf
Annual Report  
2024
A.P . Møller - Mærsk A/S 
Esplanaden 50, DK-1263 Copenhagen K | Registration no. 22756214 ALL THE WAY 

Resultat 2 (score: 0.9259):
Source: Data/Maersk Annual Report 2024.pdf
Annual Report  
2024
A.P . Møller - Mærsk A/S 
Esplanaden 50, DK-1263 Copenhagen K | Registration no. 22756214 ALL THE WAY 

Resultat 3 (score: 1.0913):
Source: Data/Maersk Annual Report 2024.pdf
means of identification
A.P . Møller - Mærsk A/S
Address of entity’s registered office
Esplanaden 50, DK-1263 Copenhagen K
 
Design and layout
e-Types
Cover photo
Ane Mærsk arriving at Port of Tanjung Pelepas, 
Malaysia, February 2024.
Produced in Denmark 2025 

Resultat 4 (score: 1.0913):
Source: Data/Maersk Annual Report 2024.pdf
means of identification
A.P . Møller - Mærsk A/S
Address of entity’s registered office
Esplanaden 50, DK-1263 Copenhagen K
 
Design and layout
e-Types
Cover photo
Ane

/var/folders/pp/lgn7jpns1mv_rf12t9s9hcnw0000gn/T/ipykernel_86751/362258311.py:28: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


Svar:  For at kategorisere Maersk's risiko er det væsentligt at belyse de forskellige faktorer, der kan have indvirkning på virksomhedens økonomiske situation.

1. Industri: Maersk er en global skibsfartvirksomhed, som er udsat for flere risici i denne branche, herunder variationer i markedets behov, konjunkturfluctueringer, økonomiske kriser og internationale regelverk.
2. Geografi: Virksomhedens stor udstrækning med opslagspladser og produktionsanlæg over hele verden betyder, at Maersk er udsat for regionale risici, herunder politiske instabiliteter, økonomiske kriser, naturlige katastrofer og handelskrig.
3. Arbejdsstyrke: Som det også beskrives i konteksten, har Maersk en stor seafarer befolkning med mere end 12.000 ansatte, hvilket betyder, at de organisatoriske og personale risici er store. Dette kan omfatte blandt andet industrielle strømeldrejser, arbejdsmarkedsfluctueringer, arbejdsmarkedslovgivning og lønningsforhold.
4. Miljø: Virksomhedens aktiviteter har en betydelig indvi